In [1]:
import karman

In [2]:
dataset=karman.ThermosphericDensityDataset(lag_minutes_omni=10, lag_days_fism2_daily=10, lag_minutes_fism2_flare=100, normalize=True)

Loading Thermospheric Density Dataset:
Loading OMNIWeb (1min) Dataset:
Loading FISM2 Daily Irradiance Dataset:
Loading FISM2 Flare (10min) Irradiance Dataset:


100%|██████████| 232/232 [01:18<00:00,  2.94it/s]



Normalizing data:

Normalizing thermospheric data, features: ['all__day_of_year__[d]', 'all__year__[y]', 'all__seconds_in_day__[s]', 'tudelft_thermo__altitude__[m]', 'tudelft_thermo__longitude__[deg]', 'tudelft_thermo__latitude__[deg]', 'tudelft_thermo__local_solar_time__[h]', 'space_environment_technologies__f107_average__', 'space_environment_technologies__f107_obs__', 'celestrack__ap_average__', 'celestrack__ap_h_0__', 'celestrack__ap_h_1__', 'celestrack__ap_h_2__', 'celestrack__ap_h_3__', 'celestrack__ap_h_4__', 'celestrack__ap_h_5__', 'celestrack__ap_h_6__', 'all__sun_right_ascension__[rad]', 'all__sun_declination__[rad]', 'all__sidereal_time__[rad]', 'space_environment_technologies__s107_obs__', 'space_environment_technologies__s107_average__', 'space_environment_technologies__m107_obs__', 'space_environment_technologies__m107_average__', 'space_environment_technologies__y107_obs__', 'space_environment_technologies__y107_average__', 'JB08__d_st_dt__[K]']

Normalizing omniweb dat

100%|██████████| 1900/1900 [01:24<00:00, 22.46it/s]



Normalizing ground truth thermospheric density, feature name: tudelft_thermo__ground_truth_thermospheric_density__[kg/m**3]


In [3]:
import torch
from torch.utils.data.sampler import SubsetRandomSampler

with open('/home/jupyter/karman-project/scripts/train_indices.txt') as f:
    train_indices = [int(line.rstrip()) for line in f]
with open('/home/jupyter/karman-project/scripts/val_indices.txt') as f:
    val_indices = [int(line.rstrip()) for line in f]
with open('/home/jupyter/karman-project/scripts/test_indices.txt') as f:
    test_indices = [int(line.rstrip()) for line in f]

print(f"Train set length: {len(train_indices)}, proportion: {len(train_indices)/len(dataset)*100} %")
print(f"Validation set length: {len(val_indices)}, proportion: {len(val_indices)/len(dataset)*100} %")
print(f"Test set length: {len(test_indices)}, proportion: {len(test_indices)/len(dataset)*100} %")

#I perform the dataset split, creating train, valid, test dataloaders:
train_sampler=SubsetRandomSampler(train_indices)
valid_sampler=SubsetRandomSampler(val_indices)
test_sampler=SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset,
                                       batch_size=1024,
                                       sampler=train_sampler,
                                       pin_memory=True,
                                       num_workers=16)
valid_loader = torch.utils.data.DataLoader(dataset,
                                       batch_size=1024,
                                       sampler=valid_sampler,
                                       pin_memory=True,
                                       num_workers=16)
test_loader = torch.utils.data.DataLoader(dataset,
                                       batch_size=1024,
                                       sampler=test_sampler,
                                       pin_memory=True,
                                       num_workers=16)

Train set length: 25032646, proportion: 66.21185018172743 %
Validation set length: 6208340, proportion: 16.421183679792605 %
Test set length: 6565743, proportion: 17.366521775114208 %


In [4]:
retval=next(iter(test_loader))#[0].shape

In [5]:
inp,out=retval


In [6]:
inp[0].shape, inp[1].shape, inp[2].shape, inp[3].shape, out.shape

(torch.Size([1024, 27]),
 torch.Size([1024, 11, 1900]),
 torch.Size([1024, 11, 1900]),
 torch.Size([1024, 11, 23]),
 torch.Size([1024]))

In [18]:
model=DensityPredictor(input_size_thermo=retval[0][0].shape[1],
                   input_size_fism2_flare=retval[0][1].shape[-1],
                   input_size_fism2_daily=retval[0][2].shape[-1],
                   input_size_omni=retval[0][3].shape[-1],
                   output_size_fism2_flare=10,
                   output_size_fism2_daily=10,
                   output_size_omni=retval[0][3].shape[-1],dropout=0)

In [12]:
out_fism2_flare=model.model_lstm_fism2_flare(inp[1])
out_fism2_daily=model.model_lstm_fism2_flare(inp[2])
out_omni=model.model_lstm_omni.forward(inp[3])
print(out_fism2_flare.shape, out_fism2_daily.shape, out_omni.shape)

torch.Size([1024, 10]) torch.Size([1024, 10]) torch.Size([1024, 23])


In [19]:
model(inp).shape

torch.Size([1024, 1])

In [7]:
from karman import FFNN
from torch import optim
#optimizer=optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.)
#model = FFNN(num_features=len(dataset[0][0]))
